In [1]:
import numpy as np
import math
np.random.seed(100)

In [2]:
class perceptron:
    w=[]
    y=0
    delta=0
    # Funcion de activacion: 1 = funcion sigmoide; 2 = funcion tangente hiperbolica
    f_activ=1
    # Tasa de aprendizaje
    n = 1
    
    # Inicializa pesos aleatorios segun la cantidad de entradas que recibira el perceptron
    # por lo general, n_entradas = entradas + bias
    def __init__(self, n_entradas):
        self.w=np.random.rand(n_entradas)

    # Funcion de activacion/transferencia
    def factivacion(self, x):
        if(self.f_activ==1):
            # funcion sigmoide
            return 1/(1+math.exp(-x))
        elif(self.f_activ==2):
            # funcion tangente hiperbolica
            return ( 2/(1+math.exp(-2*x)) ) -1
        
    def dfactivacion(self, x):
        if(self.f_activ==1):
            # funcion sigmoide
            return x*(1-x)
        elif(self.f_activ==2):
            # funcion tangente hiperbolica
            return 1-(x*x)

    def predict(self, x):
        suma=0
        for i in range(len(x)):
            suma += x[i]*self.w[i]
            
        self.y = self.factivacion(suma)
    
    def train(self, x):
        for i in range( len(self.w) ):
            self.w[i] += self.n*self.delta*self.dfactivacion(self.y)*x[i]
    


In [3]:
class rn_multicapa:
    inputLayer = []
    outputLayer = []
    vSesgo=1
    
    def __init__(self, size_input, size_inputLayer, size_outputLayer):
        # Crea size_inputLayer perceptrones en la lista de la capa de entrada
        # Inicializa cada perceptron para tener size_input pesos, mas un peso para el bias
        for p in range(size_inputLayer):
            self.inputLayer.append(perceptron(size_input+1)) 
            
        # Crea size_outputLayer perceptrones en la lista de la capa de salida
        # Inicializa cada perceptron para tener size_inputLayer pesos, mas un peso para el bias
        for r in range(size_outputLayer):
            self.outputLayer.append(perceptron(size_inputLayer+1))
            
    def predict(self, x):
        # Agrega temporalmente la entrada bias al conjunto de entrada
        x.append(self.vSesgo) 
        # Almacena temporalmente las salidas de los perceptrones de entrada
        temp=[] 
        # Calcula el output de cada perceptron en la capa de entrada y la almacena en la lista temp
        for p in range( len(self.inputLayer) ):
            self.inputLayer[p].predict(x)
            temp.append(self.inputLayer[p].y)
        
        # Con la lista temp, calcula el output de los perceptrones en la capa de salida
        temp.append(self.vSesgo)
        for p in self.outputLayer:
            p.predict(temp)
            
        temp.clear()
        x.pop() 
    
    # Recibe un conjunto de entradas x y su conjunto de salidas correctas z
    def train(self, x, z):
        self.predict(x)
        ## Calculo de deltas
        # Definimos los deltas de los perceptrones en la capa de salida
        for p in range( len(self.outputLayer) ):
            self.outputLayer[p].delta = z[p]-self.outputLayer[p].y
            
        # Definimos los deltas de los perceptrones en la capa de entrada
        for p in range( len(self.inputLayer) ):
            suma=0
            for r in range( len(self.outputLayer) ):
                suma+= self.outputLayer[r].delta*self.outputLayer[r].w[p]
            self.inputLayer[p].delta = suma
        
        ## Calculo de nuevos pesos
        # Almacena temporalmente las salidas de los perceptrones de entrada
        x.append(self.vSesgo)
        temp=[]
        for p in range( len(self.inputLayer) ):
            self.inputLayer[p].train(x)
            temp.append(self.inputLayer[p].y)
        
        temp.append(self.vSesgo)
        for p in range( len(self.outputLayer) ):
            self.outputLayer[p].train(temp)
            
        x.pop()
        temp.clear()
            

In [4]:
# Inicializa una red neuronal que recibe 3 inputs,
# tiene 4 perceptrones de entrada y 1 de salida
rn=rn_multicapa(3,6,1)

In [5]:
setX = [[1,1],
        [1,0],
        [0,1],
        [0,0]]

setZ = [0,1,1,0]

In [6]:
setX = [[1,1,1],
        [1,1,0],
        [1,0,1],
        [1,0,0],
        [0,1,1],
        [0,1,0],
        [0,0,1],
        [0,0,0]]

setZ = [[1],[0],[0],[1],[0],[1],[1],[0]]

In [7]:
def toVF(x):
    if(x>0.5):
        return 1
    else:
        return 0

In [8]:
print("Valores deseados:")
for i, s in enumerate(setX):
    print( s[0], s[1], s[2], "|", setZ[i] )

print("Valores predichos:")
print("x1 x2 | x1 XOR x2")
for s in setX:
    rn.predict(s)
    print( s[0], s[1], s[2], "|", toVF(rn.outputLayer[0].y), f" => Y = {rn.outputLayer[0].y}" )

Valores deseados:
1 1 1 | [1]
1 1 0 | [0]
1 0 1 | [0]
1 0 0 | [1]
0 1 1 | [0]
0 1 0 | [1]
0 0 1 | [1]
0 0 0 | [0]
Valores predichos:
x1 x2 | x1 XOR x2
1 1 1 | 1  => Y = 0.8806146100225263
1 1 0 | 1  => Y = 0.8640200976753712
1 0 1 | 1  => Y = 0.8771545208750116
1 0 0 | 1  => Y = 0.8581280887075359
0 1 1 | 1  => Y = 0.8689603809591092
0 1 0 | 1  => Y = 0.8441680821435775
0 0 1 | 1  => Y = 0.8636203946345988
0 0 0 | 1  => Y = 0.8359440936778172


In [9]:
# Bucle de épocas de entrenamiento.
for j in range(1000):
    for i in range(len(setX)):
        rn.train(setX[i], setZ[i])
        rn.predict(setX[i])

In [10]:
print("Valores predichos:")
print("x1 x2 | x1 XOR x2")
for s in setX:
    rn.predict(s)
    print( s[0], s[1], s[2], "|", toVF(rn.outputLayer[0].y), f" => Y = {rn.outputLayer[0].y}" )

Valores predichos:
x1 x2 | x1 XOR x2
1 1 1 | 1  => Y = 0.8717434807039636
1 1 0 | 0  => Y = 0.4470993480222398
1 0 1 | 0  => Y = 0.06186485922357961
1 0 0 | 1  => Y = 0.8262226178922458
0 1 1 | 0  => Y = 0.06151561964985546
0 1 0 | 1  => Y = 0.8120724839354327
0 0 1 | 1  => Y = 0.9554996352035499
0 0 0 | 0  => Y = 0.0640705379745476
